# __Time series data cleaning__

Usually, data scientists spend most time on data cleaning, analysis, and preparation. 

In this notebook we will reformat, look at, and prepare our dataset with the goal of using XGBoost for prediction. 

## __XGBoost__

XGBoost is not a classical time series prediction model, but it is commonly used for time series predictions. 

It is assumed you have knowledge of XGBoost and how it works!

## __Marginal CO2 emissions__

The dataset we are using is the CO2 marginal emissions data from the Australian energy market.  This is a challenging time series!

The dataset includes the marginal CO2 emissions (t CO2 / MWh) together with a time stamp (DateTime) associated to it. The marginal CO2 emissions are measured every 5 minutes.

### __Data exploration and imputation__

In [ ]:
import datetime as dt

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

#### __The dataset__

In [ ]:
df = pd.read_pickle('../data/marginal_emissions_raw.pkl')

In [ ]:
df.head()

In [ ]:
df.info()

### __Exercise 1__

- Check the datatype of the "DateTime" column of our dataframe "df".
- Change the index of our dataframe "df" to a DateTimeIndex, using our "DateTime" column.

Make sure you get rid of the original index column (starting with: 0, 1, 2, 3,...).

### __Your solution 1:__

In [ ]:
#your solution here

In [ ]:
df.info()

In [ ]:
df.isna().any()

In [ ]:
#df.to_pickle('../data/marginal_emissions_dtindex.pkl')

### __Train / test split__

It is mandatory to perform the train / test split of a dataset before any pre-analysis before training, e.g. before the identification of potential predictors. So now that we have the format right, let's do that right away.

In [ ]:
#let's see what the time range of our dataset is
print(df.index.min())
print(df.index.max())

In [ ]:
#we have plenty of data; let's go and use a bit more than a year as our test and the remaing period as our training set
#NOTE: time series forecasts request that any training is done on data that lie in the past relative to the test data

train = df[df.index < dt.datetime(2018,6,1,0,0,0)]
test = df[df.index >= (dt.datetime(2018,6,1,0,0,0))]

In [ ]:
print(train.index.min())
print(train.index.max())

print(test.index.min())
print(test.index.max())

In [ ]:
assert train.index.max() < test.index.min()

In [ ]:
train.to_pickle('../data/train.pkl')
test.to_pickle('../data/test.pkl')

In [ ]:
#now let us redefine df in order to leave the test set alone
df = train

print(df.index.min())
print(df.index.max())
df.head()

### __Exercise 2:__

Write a function which takes in our dataframe "df" and checks if the index of our dataframe entirely increments in 5 min intervals.
The output below shows a timestamp where a time interval > 5 min occurs together with the actual time period present at that time stamp:

If your fucntions finds any interval not equal to 5 min, it should return the position where this occurs (e.g. row 783) and duration of the interval at that position (e.g. row 783: 25 min). The following lines show you more useful operations to write this function:

In [ ]:
#for the purpose of demonstration, I here import a time series with a DateTimeIndex
df_dtindex = pd.read_pickle('../data/marginal_emissions_dtindex.pkl')
df_dtindex.sort_index(inplace=True)

In [ ]:
df_dtindex.head()

In [ ]:
df_dtindex.info()

In [ ]:
#see how a DateTimeIndex is made up of individual time stamp class objects
df_dtindex.index[3]

In [ ]:
type(df_dtindex.index[3])

In [ ]:
#see what happens if you perform mathematical operations in two Timestamp objects; the respective output is a Timedelta 
time_difference = df_dtindex.index[4] - df_dtindex.index[3]
time_difference

In [ ]:
type(time_difference)

In [ ]:
#the .total_seconds() method returns us any Timedelta object in seconds
time_difference.total_seconds()

### __Your solution 2:__

In [ ]:
#your solution here

In [ ]:
#%%timeit -n3 -r10
lst = gap_finder(df, 5)

In [ ]:
#%%timeit -n1 -r3
lst, _ = correct_timedelta(df, 300)

### __Exercise 3:__

If you find any such different intervals, modify our dataframe index so that it entirely increments in 5 min intervals. Use the following lines as a guide:

In [ ]:
#let's create an arbitrary dataframe
df_example = pd.DataFrame(data=[[1, 2], [3, 4], [5, 6], [9, 10]], 
                          index=[dt.datetime(2018,6,1,0,0,0),
                                dt.datetime(2018,6,1,0,0,5),
                                dt.datetime(2018,6,1,0,0,10),
                                dt.datetime(2018,6,1,0,0,20)], 
                          columns=['A', 'B'])
df_example

In [ ]:
#let's use the pandas.date_range method to create a new index
new_index = pd.date_range(start=dt.datetime(2018,6,1,0,0,0), end=dt.datetime(2018,6,1,0,0,20), periods=5)
type(new_index)

In [ ]:
#let's EXTEND the original index with our newly created DateTimeIndex; why do the NaN values occur?
df_example = df_example.reindex(new_index)
df_example

### __Your Solution 3:__

In [ ]:
#hypothetical number of 5min intervals given the max and min values of the df time range

In [ ]:
df.shape

In [ ]:
df_new = df.reindex(new_time_range)
df_new.shape

In [ ]:
df_new.head()

In [ ]:
#file_path = '../data/marginal_emissions_dtindex_nans.pkl'
#pd.to_pickle(df_new, file_path)

### __Checking for NaNs__

It is frequently the case, that machine learning models do not accept NaN values. It is therefore crucial to check your dataset for those and to clean it from them.

In [ ]:
#let's look at our dataset from BEFORE reindexing
df.isna().any()

In [ ]:
#let's look at our dataset from AFTER reindexing
df_new.isna().any()

#### __Useful visualisation of NaNs__

In [ ]:
#dark shades indicate the presence of values
#white bars indicate the presence of NaNs
plt.style.use('ggplot')

fig = sns.heatmap(df_new.isna(), yticklabels=100000)

### __Data imputation of time series__

Due to the nature of time series data, the imputation of missing values needs extra attention. The potential presence of time series trends and seasonality renders imputations using e.g. mean or median deduced from the entire dataset little useful. Instead, any imputation should rather consider the local value range around a NaN value. In the following, we will see one approach of how that works.

<img src="../images/time_series_imputation.jpg"> <br/>
source: https://www.hindawi.com/journals/mpe/2010/513810/fig9/

In [ ]:
#we use the .rolling method to calculate the rolling mean composed of always 30 subsequent values
#the .assign method adds a new column with our rolling mean to df_new, all of which we save in df_impute

df_impute = df_new.assign(rolling_mean=df_new['t CO2-e / MWh'].rolling(window=30, min_periods=1).mean())

In [ ]:
df_impute.head(50)

In [ ]:
df_impute.isna().any()

In [ ]:
#here we use the numpy.where() method to replace only NaNs with values from the rolling_mean column
df_impute['t CO2-e / MWh'] = np.where(df_impute['t CO2-e / MWh'].isnull(),\
                                  df_impute['rolling_mean'], df_impute['t CO2-e / MWh'])

df_impute.drop(columns='rolling_mean', inplace=True)

In [ ]:
#let's see if everything worked
df_impute.isna().any()

In [ ]:
pd.set_option('display.max_rows', None)
df_impute[df_impute['t CO2-e / MWh'].isna()]

In [ ]:
df_impute.to_pickle('../data/train_clean.pkl')